# Usage for Shape distance using the WFR package

In [1]:
import PLShapes
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import time

Import PL shapes for distance computation

In [2]:
cat0=PLShapes.PLShape(r"data\cat0.ply").downsampleShape(1000)
cat1=PLShapes.PLShape(r"data\cat1.ply").downsampleShape(1000)
cat1.mesh.translate((100,0,0))

totalMass0=np.sum(cat0.meas.masses)
totalMass1=np.sum(cat1.meas.masses)

cat0.normalColorize(False)
cat1.shadeColorize(np.array([0, 0.4470, 0.7410]),False)
o3d.visualization.draw_geometries([cat0.mesh,cat1.mesh])

print("The surface area cat0 is: {}".format(totalMass0))
print("The surface area cat1 is: {}".format(totalMass1))

The surface area cat0 is: 19588.09349754554
The surface area cat1 is: 20117.263055936543


Choose parameters for the distance computation.

In [3]:
NoIterations=1000  # Maximum number of iterations before the algorithm terminates
eps = 1e-6         # Minimum relative increase the algorithm terminates

Now we will compute the SRNF distance using Algorithm 1 from *The Square Root Normal Field Metric and Unbalanced Optimal Transport*.

In [4]:
timestart = time.time();
dist,cost,ind,P,Q=cat0.shapeDistance(cat1,NoIterations,eps)
totaltime=time.time()-timestart

print("The distance is computed to be: {}+{}-2{}={}".format(totalMass0,totalMass1,cost[ind,0],dist.cpu()[0]))
print("The algorithm took {} iterations to terminate.".format(ind))
print("The algorithm took {} seconds to converge.".format(totaltime))

The distance is computed to be: 19588.09349754554+20117.263055936543-219286.421384220732=33.652842153978874
The algorithm took 320 iterations to terminate.
The algorithm took 1.6858668327331543 seconds to converge.


Now we can plot correspondences given by the optimal semi-coupling.

In [5]:
cat1.colorizeFromMap(P,cat0.meas.supports,False)
o3d.visualization.draw_geometries([cat0.mesh,cat1.mesh])

Choose more parameters for the distance computation mod SO3.

In [6]:
cat0.normalColorize(False)
cat1.shadeColorize(np.array([0, 0.4470, 0.7410]),False)
RotIterations=100
RotDepth=100

We can also perform rotational alignment using this method.

In [7]:
timestart = time.time();
dist,P,Q,R=cat0.shapeDistanceModSO3(cat1,RotIterations,RotDepth,NoIterations,eps)
totaltime=time.time()-timestart

print("The distance is computed to be: {}".format(dist.cpu()[0]))
print("The algorithm took {} iterations to terminate.".format(ind))
print("The algorithm took {} seconds to converge.".format(totaltime))

The distance is computed to be: 17.47785356210148
The algorithm took 320 iterations to terminate.
The algorithm took 5.194364786148071 seconds to converge.


Now we can plot correspondences given by the optimal semi-coupling and the rotational alignment produced by this method.

In [10]:
ncat1=PLShapes.PLShape(r"data\cat1.ply").downsampleShape(1000)
ncat1.mesh.translate((200,0,0))
ncat1=ncat1.getRotatedShape(R)
ncat1.colorizeFromMap(P,cat0.meas.supports,False)

o3d.visualization.draw_geometries([cat0.mesh,cat1.mesh,ncat1.mesh])

# Complete Documentation of PLShapes.py below

In [11]:
help(PLShapes)

Help on module PLShapes:

NAME
    PLShapes

CLASSES
    builtins.object
        PLShape
    
    class PLShape(builtins.object)
     |  PLShape(filename=None, mesh=None, meas=None)
     |  
     |  Methods defined here:
     |  
     |  __init__(self, filename=None, mesh=None, meas=None)
     |      Init        
     |      ----------
     |      PLShape Object
     |          returns a Shape Object itialized from a PLY file, a Open3D mesh, or a Measure Object
     |  
     |  colorizeFromMap(self, P, N, visualize)
     |      Colors the PL shape so that each face is shaded according semi-coupling P,
     |      
     |      Parameters
     |      ----------
     |      P : tensor 
     |          The semi-coupling that defines the map from the PL shape to some other PL shape
     |      ----------
     |      N : np.array 
     |          The unit normal vectors of the other PL shape
     |      ----------
     |      visualize : boolean
     |          whether or not to visualize th